# 第4章 条件判断

## 4.1 判断运算符

对于所以自定义类，默认都为True，当然也可以通过自定义`__bool__`魔术方法，定制判定逻辑。

对于`None True False`（单例模式），要使用`is`关键字，而不是`==`；前者关注的是`id()`，而后者关注的是对象的值是否相等。

对于自定义类来说，在进行`==`判断时，可通过实现`__eq__`魔术方法。



In [1]:
class EqualWithAnything:
    """与任何对象相等"""
    
    def __eq__(self, other):
        # other 为实例比较时 == 操作符的 右边的对象
        return True

`and` 和 `or` 的运算优先级不一样；`and`优先级高于`or`;且 `or`是短路运算符。

In [1]:
(True or False) and False

False

In [3]:
True or False and False

True

在实践中，我们应该留意`all()`和`any()`内置函数：接收一个可迭代对象，并返回对应的bool值。因此有些代码就可以更加简洁和符合直觉：

In [17]:
def all_numbers_gt_10(numbers):
    for n in numbers:
        if n <= 10:
            return False
    return True
    
all(n<=10 for n in numbers)

False

## 4.2 范围类分支优化

我们经常会遇到多个写分支的情况，比如说将学生成绩转换为ABCD等级，正常的写法，可能就是写一些条件判断语句。

但是针对此场景，还有更加优化的写法，那就是使用`bisect`内置模块。`bisect.bisect(a, x)`此模块操作的容器a一定是*有序的*，返回下标i，表示将元素x按照i的位置插入容器，并不会破坏原有容器的有序性。

基于此特性，我们可以优化范围类分支的书写方式。可能有个疑问，如果待分类数字刚好是处于边界上，那么如何判定呢？

- bisect_left 将x插入到a中相同元素的左边
- bisect_right 将x插入到a中相同元素的右边
- bisect  将x插入到a中相同元素的右边

> 个人感觉如果不会有边界判定不一致的情况，且需求变动不大的时候可以用。总之 可有可无的一种特性。

In [33]:
import bisect 

def get_score_rank(int_score):
    score_point = (60,70,80,90)
    rank = bisect.bisect_right(score_point, int(int_score))
    
    grades = ('E','D','C','B','A')
    return grades[rank]

get_score_rank(60)

'D'

## 4.3 降低分支相似性

我们应该充分利用“一等公民”的特性，无感的应用工厂模式，提高代码可读性，参考如下代码，及优化后的样子：

In [ ]:
# 优化前
def before_foo(data):
    if user.exist:
        create_user_profile(
            username = data.username,
            gender = data.gender,
            email = data.email,
            age = data.age,
            address = data.address,
            pointers = 0,
            created = now(),
        )
    else:
        update_user_profile(
            username = data.username,
            gender = data.gender,
            email = data.email,
            age = data.age,
            address = data.address,
            updated = now(),
        )
        
# 优化后, 更能感知两者的差异性。
def after_foo(data):
    if user.exist:
        _update_or_create = create_user_profile
        exter_args = {'pointers': 0, 'created': now()}
    else:
        _update_or_create = update_user_profile
        exter_args = {'updated': now()}

    _update_or_create(
        username = data.username,
        gender = data.gender,
        email = data.email,
        age = data.age,
        address = data.address,
        **exter_args,
    )
    

# 第5章 异常与错误处理

Python编程有两种风格，一种是LBYL一种是EAFP；前者是预先想各种条件判断，后者是不做任何事前检查，直接执行操作，并用`try`捕获异常。毕竟我们的目的是执行目的，而不是每次check 边界条件。

Python社区更加喜爱EAFP风格编码，每次直觉驱使写if/else 进行错误判断时，考虑使用`try`的 EAFP 风格。Pythonista更喜欢EAFP。

比如以下两种风格的代码：

In [3]:
# LBYL
def add_one(value):
    if isinstance(value, int):
        return value + 1
    elif isinstance(value, str) and value.isdigit():
        return int(value) + 1
    else:
        raise TypeError("错误")

# EAFP
def foo_add_one(value):
    try:
        return value + 1
    except (TypeError, ValueError) as e:
        print(f'Unable to perform incr for value:"{value}", error: {e}') 
    finally:
        print('')

foo_add_one([11])

Unable to perform incr for value:"[11]", error: can only concatenate list (not "int") to list



## 5.1 try

1. 父类异常靠后捕获，优先清晰异常；
2. else 分支，只在没有异常时执行，但是如果有`return`或者`break`也不会执行；`finally`以上情况也会执行。
3. 空`raise`语句，可以抛出异常，交由上层捕获。


In [13]:
def incr_by_key(d, key):
    try:
        d[key] += 1
    except TypeError:
        print('表面上我捕获了')
        raise

def main():
    str_dict = {"my":"rohan"} 
    try:
        incr_by_key(str_dict, 'my')
    except TypeError:
        print("其实我还是往上抛了")
        
main()

表面上我捕获了
其实我还是往上抛了


## 5.2 自定义异常 

对于函数中的异常，它不像是返回值，它在被捕获前会层层上报，这个特性让代码更加灵活，但也带来了更大的风险。如果缺少一个顶级的统一异常处理逻辑，某个忽视的异常会弄垮这个程序。

In [33]:
class CreateItemError(Exception):
    """创建Item失败"""
    
def create_itme(name):
    """创建新的Item
    :raise: 当无法创建时抛出CreateItemError
    """
    if True:
        raise CreateItemError('item err')
        
create_itme(33)

CreateItemError: item err

## 5.3 上下文管理器减少样本代码

上下文管理器定义了“进入”和“退出”动作的特殊对象：`__enter__`和`__exit__`； 其中`__enter__`魔术方法返回的对象即`with as object`中的`object`

`with`语句和`try`结合，减少的样本代码如下：

1. 替代finally
2. 忽略已知异常

### 5.3.1 替代finally

`finally`经常用作资源清理类工作，比如关闭数据库连接。我们就可以使用 `with`替代。

其中`__exit__`接收的三个参数：`(exc_type, exc_value, traceback)`，如果`with`语句中没有报错，那么这三个参数都为空。如果报错：
- exc_type：异常类型
- exc_value：异常对象
- traceback：错误的堆栈对象


此时，如果`__exit__` 返回 `True`那么将忽略异常；如果返回`False`，那么异常正常抛出，需要在`with`代码中写`try`捕获。

In [ ]:
# 正常写法
def create_conn(*args, **kwargs):
    pass

conn = create_conn('127.0.0.1', '8080', timeout=None)
try:
    conn.send_text("hello")
except Exception as e:
    print(e)
finally:
    conn.close()
    
    
    
# 写一个 上下文管理器
class CreateConn:
    def __init__(self, host, port, timeout=None):
        self.conn = create_conn(host, port, timeout=None)
        
    def __enter__(self):
        return self.conn
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        # 如果有异常不忽略
        return False
    
with CreateConn(host, port, timeout=None) as conn:
    try:
        conn.send_text('hello')
    except Exception as e:
        print(e)

注意，此处的“`with`语句中没有报错”，指的是外部代码，with语句下的代码。在`__exit__` 中报错的代码不会在这里被捕获。

In [32]:
class MyWith:
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        return self.name
    
    def __exit__(self, exc_type, exc_value, traceback):
        # 如果有异常不忽略
        print(exc_type,exc_value, traceback )
        if exc_type == AttributeError:
            return True
        else:
            return False
    
with MyWith('啊') as test:
    raise AttributeError
    # 抛出异常后，直接exit了，后面的代码并没有运行。
    print(test)

<class 'AttributeError'>  <traceback object at 0x107dda1c0>


### 5.3.2 contextmanger 装饰器

为了简化with上下文管理工具，Python提供了一个装饰器`contextmanger`来简化此工作。`@contextmanger`装饰器位于内置模块`contextlib`下，它可以把任何一个生成器函数直接转换成上下文管理器。

被装饰的函数在被调用时，必须返回一个 generator 迭代器。 这个迭代器必须只 `yield` 一个值出来，这个值会被用在 `with` 语句中，绑定到 `as` 后面的变量，如果给定了的话。

当生成器发生 `yield` 时，嵌套在 `with` 语句中的语句体会被执行。 语句体执行完毕离开之后，该生成器将被恢复执行。 如果在该语句体中发生了未处理的异常，则该异常会在生成器发生 yield 时重新被引发。 因此，你可以使用 try...except...finally 语句来捕获该异常（如果有的话），或确保进行了一些清理。 如果仅出于记录日志或执行某些操作（而非完全抑制异常）的目的捕获了异常，生成器必须重新引发该异常。 否则生成器的上下文管理器将向 with 语句指示该异常已经被处理，程序将立即在 with 语句之后恢复并继续执行。

In [42]:
from contextlib import contextmanager

def foo(*args, **kwds):
    print("我是foo函数里面")
    return [1, 2, 3]

@contextmanager
def managed_resource(*args, **kwds):
    # Code to acquire resource, e.g.:
    print("类似于类构造方法中的__init__")
    resource = foo(*args, **kwds)
    try:
        print("类似于类构造方法中的__enter__")
        yield resource
        print("类似于类构造方法中的__exit__")
    except TypeError as e:
        print("对外部循环体的异常进行处理")
    finally:
        # Code to release resource, e.g.:
        print(" Code to release resource")

with managed_resource(timeout=3600) as resource:
    print("这是外部循环体")
    raise TypeError

类似于类构造方法中的__init__
我是foo函数里面
类似于类构造方法中的__enter__
这是外部循环体
对外部循环体的异常进行处理
 Code to release resource


## 5.4 自定义异常

自定义异常不需要遵守太多规范：
1. 继承`Exception`，而不是`BaseException`;
2. 异常类名以`Error`或者`Exception`结尾;
3. 保证调用方能够清晰区分各种异常。

其他小技巧：
1. 自定义异常类时，可以用类之间的继承，从而设计更加精准的异常子类；
2. 可以创建包含“错误代码”等额外属性的异常类。


In [45]:
class CreateItemError(Exception):
    """创建Item失败
    :param error_code: 错误代码
    :param message: 错误信息
    """
    
    def __init__(self, error_code, message):
        self.error_code = error_code
        self.message = message
        super().__init__(f'{self.error_code}-{self.message}')

class CreateItemFullError(CreateItemError):
    """当前Item已满"""

    
raise CreateItemFullError("Item_Full", "Item_Full")
raise CreateItemError("name_too_long", "toooooo long")

CreateItemFullError: Item_Full-Item_Full

## 5.5 异常何必是异常

空对象模式（null object pattern），简单来说就是你写的某个API，在本该返回`None`或者抛出异常时，返回一个结构与正常结果一致的，一个特定的“空类型对象”来代替，以免去其他人调用时，需要`try`捕获异常。

In [46]:
QUALIFIED_POINTS = 80

class UserPoint:
    """用户得分纪录"""
    
    def __init__(self, username, points):
        self.username = username
        self.points = points
    
    def is_qualified(self):
        return self.points >= QUALIFIED_POINTS
    

class NullUserPoint(UserPoint):
    """创建用户失败的时候"""
    
    def __init__(self):
        self.username = ""
        self.points = 0
        super.__init__(self.username, self.points)
        
    def is_qualified(self):
        return False
    


## 5.6 数据校验

某个函数，在执行前，对输入可能有苛刻的校验条件，这就导致了大量的判定语句。我们可以使用`pydantic`库，此库非官方自带库，需要pip安装。

不要使用`assert`语句作校验，这是供开发者调试程序的关键字，使用`-O`标志运行代码，那么这些校验都将被跳过。

优点：
- 快
- 验证复杂结构：递归
- 可扩展自定义类型
- @dataclass 装饰器

In [10]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError


class User(BaseModel):
    # 指定id为int，如果不是强制到ints；否则将引发异常
    id: int
    # name 不提供则为None，根据类型判断为 str
    name = 'John Doe'
    # signup_ts 可选的日期时间字段
    signup_ts: Optional[datetime] = None
    # 全部为int类型的list
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}
user = User(**external_data)
print("user.id: ",user.id)


external_data['friends'].append("str")
try:
    user = User(**external_data)
except ValidationError as e:
    print(e.json())


user.id:  123
[
  {
    "loc": [
      "friends",
      3
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]
